# Scale Detection Pipeline - Complete Implementation

This notebook implements the complete scale detection pipeline following the Uni-AIMS paper architecture:

1. **Dataset Conversion**: Convert JSON annotations to YOLO format
2. **Model Training**: Train YOLOv8m for joint detection of scale bars and text regions
3. **Endpoint Localization**: Refine scale bar endpoints for accurate pixel length measurement
4. **OCR Processing**: Extract and parse scale text using PaddleOCR
5. **Scale Matching**: Match text regions to their corresponding scale bars
6. **Pixel-to-Physical Conversion**: Convert pixel measurements to physical units
7. **Evaluation**: Comprehensive evaluation of the entire pipeline

## Requirements

Make sure you have installed all required packages:
```bash
pip install ultralytics paddlepaddle paddleocr opencv-python scikit-image scipy matplotlib seaborn pandas scikit-learn
```


## 1. Setup and Imports


In [8]:
import os
import sys
import json
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import cv2
import warnings
warnings.filterwarnings('ignore')

# Add src directory to path
sys.path.append('src')

# Import our custom modules
import src.get_data as get_data
import src.convert_jsons_to_yolo as convert_jsons
import src.train_yolo as train_model
from src.postprocess_scalebar import localize_scale_bar_endpoints, visualize_endpoint_detection
from src.ocr_and_match import ScaleDetectionPipeline, ScaleBarDetection, TextDetection
from src.pixels_to_mm import PixelToPhysicalConverter, ScaleInfo, create_converter_from_matches
from src.evaluate_pipeline import ComprehensiveEvaluator, EvaluationMetrics

# Auto-reload modules
%load_ext autoreload
%autoreload 2

# Set up plotting
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# Step 1: Get the data
DATA_DIR = Path('data')
IMAGES_DIR = DATA_DIR / "images"
JSONS_DIR = DATA_DIR / "jsons"

# Check if data directories exist, if not download the dataset
if not (
    os.path.exists(DATA_DIR) and 
    os.path.exists(IMAGES_DIR) and 
    os.path.exists(JSONS_DIR)
):
    get_data.download_from_path('original1/scalebar-dataset', DATA_DIR, IMAGES_DIR, JSONS_DIR)

In [3]:
# Define directory paths
DATA_DIR = Path('data')
IMAGES_DIR = DATA_DIR / "images"
JSONS_DIR = DATA_DIR / "jsons"
LABELS_DIR = DATA_DIR / "labels"

# Check if already converted
if os.path.exists(LABELS_DIR) and os.listdir(LABELS_DIR):
    print(f"Labels directory {LABELS_DIR} already exists and is not empty. Skipping conversion.")
    yaml_path = DATA_DIR / 'data.yaml'
else:
    os.makedirs(LABELS_DIR, exist_ok=True)

    # Count files
    image_files = list(Path(IMAGES_DIR).glob("*.jpg"))
    json_files = list(Path(JSONS_DIR).glob("*.json"))

    print(f"Found {len(image_files)} image files")
    print(f"Found {len(json_files)} JSON annotation files")

    # Convert dataset
    yaml_path = convert_jsons.convert_dataset(DATA_DIR, 0.8)

    # Run validation if requested
    if True:
        stats = convert_jsons.validate_conversion(
            LABELS_DIR / 'train',
            sample_size=-1
        )
        print(f"Validation results:")
        for key, value in stats.items():
            print(f"  {key}: {value}")

Labels directory data/labels already exists and is not empty. Skipping conversion.


In [5]:
args = {
    "data_yaml": 'data_small/data.yaml',  # Path to data.yaml file
    "model_name": "yolov8m.pt",  # Pretrained model name from ultralytics
    "epochs": 50,
    "imgsz": 1280,
    "batch": 4,
    "lr0": 0.01,
    "weight_decay": 0.0005,
    "warmup_epochs": 3,
    "patience": 10,
    "save_period": 10,
    "device": 'mps',  # Use 'cpu' or GPU id like '0'
    "model_dir": "models",
    "name": "train",
    "resume": False,  # Whether to resume from last training
    "export": True,  # Whether to export the model after training
    "export_format": 'onnx',  # Format to export: 'onnx', 'torchscript', etc.
    "export_name": 'scalebar_detector.onnx',
    "clearml": True,  # Whether to use ClearML for experiment tracking
    "clearml_project": "scalebar_detection",  # ClearML project name
}

# Optional: initialize ClearML Task
task = None
if args['clearml']:
    try:
        from clearml import Task
        task = Task.init(
            project_name=args['clearml_project'],
        )
    except Exception as e:
        print(f"Warning: ClearML not initialized ({e}). Continue without tracking.")
        task = None

# Create model directory if it doesn't exist
MODEL_DIR = Path(args['model_dir'])
os.makedirs(MODEL_DIR, exist_ok=True)

# Train model
print("Starting YOLOv8 training...")
model, results = train_model.train_model(
    data_yaml=args['data_yaml'],
    model_name=args['model_name'],
    epochs=args['epochs'],
    imgsz=args['imgsz'],
    batch=args['batch'],
    lr0=args['lr0'],
    weight_decay=args['weight_decay'],
    warmup_epochs=args['warmup_epochs'],
    patience=args['patience'],
    save_period=args['save_period'],
    device=args['device'],
    model_dir=MODEL_DIR,
    name=args['name'],
    resume=args['resume'],
)

print("Training completed!")

# Determine run directory from trainer if available
run_dir = None
try:
    run_dir = getattr(model.trainer, 'save_dir', None)
except Exception:
    run_dir = None
if not run_dir:
    # Fallback to expected structure
    run_dir = os.path.join(MODEL_DIR, args.name)
run_dir = str(run_dir)

# Export model if requested
if args['export']:
    print("Exporting model...")
    exported_path = train_model.export_model(
        model, 
        os.path.join(MODEL_DIR, args['export_name']), 
        format=args['export_format']
    )
else:
    exported_path = None

# Log artifacts to ClearML
if task is not None:
    try:
        # Connect basic hyperparameters
        overrides = train_model.create_training_config(
            data_yaml=args['data_yaml'],
            model_name=args['model_name'],
            epochs=args['epochs'],
            imgsz=args['imgsz'],
            batch=args['batch'],
            lr0=args['lr0'],
            weight_decay=args['weight_decay'],
            warmup_epochs=args['warmup_epochs'],
            patience=args['patience'],
            project=args['model_dir'],
            name=args['name'],
            resume=args['resume'],
        )
        task.connect(overrides)

        # Common artifacts
        results_csv = os.path.join(run_dir, 'results.csv')
        best_weights = os.path.join(run_dir, 'weights', 'best.pt')
        last_weights = os.path.join(run_dir, 'weights', 'last.pt')

        if os.path.exists(results_csv):
            task.upload_artifact('results.csv', artifact_object=results_csv)
        if os.path.exists(best_weights):
            task.upload_artifact('best.pt', artifact_object=best_weights)
        if os.path.exists(last_weights):
            task.upload_artifact('last.pt', artifact_object=last_weights)
        if exported_path and os.path.exists(exported_path):
            task.upload_artifact(Path(exported_path).name, artifact_object=exported_path)
    except Exception as e:
        print(f"Warning: failed to upload ClearML artifacts: {e}")

Starting YOLOv8 training...
2025-09-24 17:17:36,104 - clearml.model - INFO - Selected model id: e10d5ac9d5374035ae82b310f547e378
Ultralytics 8.3.203 🚀 Python-3.11.13 torch-2.5.1 MPS (Apple M4 Pro)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data_small/data.yaml, degrees=0.0, deterministic=True, device=mps, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=1280, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8m.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=N

KeyboardInterrupt: 

## 3. YOLOv8 Model Training


In [3]:
%env CLEARML_WEB_HOST=https://app.clear.ml/
%env CLEARML_API_HOST=https://api.clear.ml
%env CLEARML_FILES_HOST=https://files.clear.ml
%env CLEARML_API_ACCESS_KEY=X6DE0NND8TP2KDWLEZBYL3TFVJSBA1
%env CLEARML_API_SECRET_KEY=mD7N9_8a35gxuXvd_e12B5GcZ8KrBWgqohi-ZB5LKx8kDUTqfvOvodZwr4FGM08uyfU

env: CLEARML_WEB_HOST=https://app.clear.ml/
env: CLEARML_API_HOST=https://api.clear.ml
env: CLEARML_FILES_HOST=https://files.clear.ml
env: CLEARML_API_ACCESS_KEY=X6DE0NND8TP2KDWLEZBYL3TFVJSBA1
env: CLEARML_API_SECRET_KEY=mD7N9_8a35gxuXvd_e12B5GcZ8KrBWgqohi-ZB5LKx8kDUTqfvOvodZwr4FGM08uyfU


In [4]:
import torch
torch.mps.empty_cache()

In [ ]:
import src

Starting YOLOv8 training...
New https://pypi.org/project/ultralytics/8.3.203 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.197 🚀 Python-3.11.13 torch-2.5.1 MPS (Apple M4 Pro)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data_small/data.yaml, degrees=0.0, deterministic=True, device=mps, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=1280, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8m.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=

In [ ]:
# Predict and visualize some results
# Load the best model
from ultralytics import YOLO
best_model_path = 'models/train/weights/best.pt'  # Update with actual path if needed
model = YOLO(best_model_path)
for i in range(1, 1000):
    try:
        image_path = f"data/images/train/{i}.jpg"
        results = model.predict(image_path)

        # Visualize results
        results[0].plot()
        plt.show()
    except Exception as e:
        print(f"Error processing image {i}: {e}")


image 1/1 /Users/emmaboehly/Documents/Vector/ScaleDetection/data/images/train/1.jpg: 928x1280 (no detections), 1071.8ms
Speed: 4.9ms preprocess, 1071.8ms inference, 1.1ms postprocess per image at shape (1, 3, 928, 1280)

image 1/1 /Users/emmaboehly/Documents/Vector/ScaleDetection/data/images/train/2.jpg: 1024x1280 (no detections), 1274.9ms
Speed: 7.9ms preprocess, 1274.9ms inference, 1.7ms postprocess per image at shape (1, 3, 1024, 1280)

image 1/1 /Users/emmaboehly/Documents/Vector/ScaleDetection/data/images/train/3.jpg: 1280x1248 (no detections), 1499.8ms
Speed: 7.1ms preprocess, 1499.8ms inference, 1.0ms postprocess per image at shape (1, 3, 1280, 1248)

image 1/1 /Users/emmaboehly/Documents/Vector/ScaleDetection/data/images/train/4.jpg: 1184x1280 (no detections), 1509.6ms
Speed: 10.3ms preprocess, 1509.6ms inference, 0.8ms postprocess per image at shape (1, 3, 1184, 1280)

image 1/1 /Users/emmaboehly/Documents/Vector/ScaleDetection/data/images/train/5.jpg: 864x1280 (no detections

KeyboardInterrupt: 

## 4. Endpoint Localization Testing


In [14]:
from src.utils import points_to_xywh

# Test endpoint localization on a sample image
sample_id = 1
IMAGES_DIR = os.path.join("data/images")
JSONS_DIR = os.path.join("data/jsons")
OUTPUT_DIR = os.path.join("output")

# Load a sample image
sample_image_path = os.path.join(IMAGES_DIR, "train/{}.jpg".format(sample_id))
json_path = os.path.join(JSONS_DIR, f"{sample_id}.json")

if os.path.exists(sample_image_path):
    # Load image
    image = cv2.imread(sample_image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Predict using the trained model
    #results = model.predict(source=sample_image_path, imgsz=1280, conf=0.25, device='mps')
    #boxes = results[0].boxes.xyxy.cpu().numpy()    # Bounding boxes
    #scores = results[0].boxes.conf.cpu().numpy()   # Confidence scores
    #classes = results[0].boxes.cls.cpu().numpy()   # Class IDs

    # Load corresponding JSON annotation
    with open(json_path, 'r') as f:
        data = json.load(f)
    
    # Test endpoint localization for each scale bar
    for i, bar in enumerate(data.get('bars', [])):
        if 'points' in bar and len(bar['points']) >= 2:
            # Calculate bounding box from points
            bbox = points_to_xywh(bar['points'])

            print(f"\nProcessing scale bar {sample_id}, {i+1}:")
            print(f"  Bounding box: {bbox}")
            
            # Localize endpoints
            result = localize_scale_bar_endpoints(image, bbox, radius=100)
            
            if result['success']:
                print(f"  Endpoints: {result['endpoints']}")
                print(f"  Pixel length: {result['pixel_length']:.2f}")
                
                # Visualize result
                vis_save_path = os.path.join(OUTPUT_DIR, f"endpoint_detection_{sample_id}_{i+1}.png")
                visualize_endpoint_detection(image, bbox, result, vis_save_path)
            else:
                print(f"  Failed: {result['error']}")
else:
    print("Sample files not found. Skipping endpoint localization test.")



Processing scale bar 1, 1:
  Bounding box: (387, 462, 263, 6)
Processing ROI at (387, 462), size (263x6)
ROI shape: (6, 263, 3)
ROI: [[[217 217 217]
  [220 220 220]
  [223 223 223]
  ...
  [220 220 220]
  [215 215 215]
  [222 222 222]]

 [[135 135 135]
  [136 136 136]
  [144 144 144]
  ...
  [140 140 140]
  [140 140 140]
  [152 152 152]]

 [[144 144 144]
  [142 142 142]
  [145 145 145]
  ...
  [141 141 141]
  [144 144 144]
  [148 148 148]]

 [[142 142 142]
  [140 140 140]
  [143 143 143]
  ...
  [163 163 163]
  [148 148 148]
  [152 152 152]]

 [[150 150 150]
  [149 149 149]
  [150 150 150]
  ...
  [136 136 136]
  [143 143 143]
  [148 148 148]]

 [[145 145 145]
  [144 144 144]
  [145 145 145]
  ...
  [160 160 160]
  [148 148 148]
  [154 154 154]]]

Selected best channel: (6, 263)
Best channel: [[217 220 223 ... 220 215 222]
 [135 136 144 ... 140 140 152]
 [144 142 145 ... 141 144 148]
 [142 140 143 ... 163 148 152]
 [150 149 150 ... 136 143 148]
 [145 144 145 ... 160 148 154]]

Applied

<Figure size 1500x1000 with 6 Axes>

## 5. OCR and Scale Matching Pipeline


In [70]:
# Initialize OCR and matching pipeline
print("Initializing OCR and matching pipeline...")

pipeline = ScaleDetectionPipeline(
    ocr_backend='paddle',  # or 'easyocr', 'tesseract'
    confidence_threshold=0.15,
    max_distance_ratio=1.5
)

Creating model: ('PP-LCNet_x1_0_doc_ori', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/Users/emmaboehly/.paddlex/official_models/PP-LCNet_x1_0_doc_ori`.
Creating model: ('UVDoc', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/Users/emmaboehly/.paddlex/official_models/UVDoc`.
Creating model: ('PP-LCNet_x1_0_textline_ori', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/Users/emmaboehly/.paddlex/official_models/PP-LCNet_x1_0_textline_ori`.


Initializing OCR and matching pipeline...


Creating model: ('PP-OCRv5_server_det', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/Users/emmaboehly/.paddlex/official_models/PP-OCRv5_server_det`.
Creating model: ('en_PP-OCRv5_mobile_rec', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/Users/emmaboehly/.paddlex/official_models/en_PP-OCRv5_mobile_rec`.


In [71]:
# Test OCR and matching on sample images
print("Testing OCR and matching on sample images...")
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Process first 5 images
sample_results = []
for i in range(1, 20):
    image_path = os.path.join(IMAGES_DIR, f"train/{i}.jpg")
    json_path = os.path.join(JSONS_DIR, f"{i}.json")
    
    if os.path.exists(image_path) and os.path.exists(json_path):
        print(f"\nProcessing image {i}...")
        
        # Load image
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Load annotations and create scale bar detections
        with open(json_path, 'r') as f:
            data = json.load(f)
        
        label_detections = []
        for label in data.get('labels', []):
            if 'points' in label and len(label['points']) >= 2:
                points = np.array(label['points'])
                x_min, y_min = np.min(points, axis=0)
                x_max, y_max = np.max(points, axis=0)
                
                bbox = (int(x_min), int(y_min), int(x_max - x_min), int(y_max - y_min))
                center = ((x_min + x_max) / 2, (y_min + y_max) / 2)
                
                label_detections.append(ScaleBarDetection(
                    bbox=bbox,
                    center=center,
                    confidence=0.5,
                    pixel_length=np.sqrt((x_max - x_min)**2 + (y_max - y_min)**2)
                ))
        
        # Process with pipeline
        results = pipeline.process_image(image, label_detections)
        
        print(f"  Text detections: {results['total_text_detections']}")
        print(f"  Scale bar detections: {results['total_bar_detections']}")
        print(f"  Successful matches: {results['successful_matches']}")
        
        # Show matched scales
        for j, match in enumerate(results['matches']):
            print(f"    Match {j+1}: '{match.text.text}' -> {match.text.parsed_value} {match.text.normalized_unit}")
            if match.um_per_pixel:
                print(f"      Scale: {match.um_per_pixel:.6f} um/pixel")
        
        sample_results.append(results)
        
        # Save results
        output_path = os.path.join(OUTPUT_DIR, f"ocr_results_{i}.json")
        pipeline.save_results(results, output_path)

print(f"\nProcessed {len(sample_results)} images successfully!")


[2025-09-22 19:05:36,454] [ WARNING] ocr_and_match.py:28 - OCR detection failed: tuple index out of range
[2025-09-22 19:05:36,478] [ WARNING] ocr_and_match.py:28 - OCR detection failed: tuple index out of range
[2025-09-22 19:05:36,497] [ WARNING] ocr_and_match.py:28 - OCR detection failed: tuple index out of range
[2025-09-22 19:05:36,527] [ WARNING] ocr_and_match.py:28 - OCR detection failed: tuple index out of range
[2025-09-22 19:05:36,545] [ WARNING] ocr_and_match.py:28 - OCR detection failed: tuple index out of range
[2025-09-22 19:05:36,563] [ WARNING] ocr_and_match.py:28 - OCR detection failed: tuple index out of range
[2025-09-22 19:05:36,593] [ WARNING] ocr_and_match.py:28 - OCR detection failed: tuple index out of range


Testing OCR and matching on sample images...

Processing image 1...
  Text detections: 0
  Scale bar detections: 0
  Successful matches: 0
Results saved to: output/ocr_results_1.json

Processing image 2...
  Text detections: 0
  Scale bar detections: 1
  Successful matches: 0
Results saved to: output/ocr_results_2.json

Processing image 3...
  Text detections: 0
  Scale bar detections: 0
  Successful matches: 0
Results saved to: output/ocr_results_3.json

Processing image 4...
  Text detections: 0
  Scale bar detections: 1
  Successful matches: 0
Results saved to: output/ocr_results_4.json

Processing image 5...
  Text detections: 0
  Scale bar detections: 0
  Successful matches: 0
Results saved to: output/ocr_results_5.json

Processing image 6...
  Text detections: 0
  Scale bar detections: 1
  Successful matches: 0
Results saved to: output/ocr_results_6.json

Processing image 7...
  Text detections: 0
  Scale bar detections: 1
  Successful matches: 0
Results saved to: output/ocr_resu

[2025-09-22 19:05:36,610] [ WARNING] ocr_and_match.py:28 - OCR detection failed: tuple index out of range
[2025-09-22 19:05:36,628] [ WARNING] ocr_and_match.py:28 - OCR detection failed: tuple index out of range
[2025-09-22 19:05:36,649] [ WARNING] ocr_and_match.py:28 - OCR detection failed: tuple index out of range
[2025-09-22 19:05:36,677] [ WARNING] ocr_and_match.py:28 - OCR detection failed: tuple index out of range
[2025-09-22 19:05:36,699] [ WARNING] ocr_and_match.py:28 - OCR detection failed: tuple index out of range
[2025-09-22 19:05:36,719] [ WARNING] ocr_and_match.py:28 - OCR detection failed: tuple index out of range
[2025-09-22 19:05:36,737] [ WARNING] ocr_and_match.py:28 - OCR detection failed: tuple index out of range
[2025-09-22 19:05:36,755] [ WARNING] ocr_and_match.py:28 - OCR detection failed: tuple index out of range
[2025-09-22 19:05:36,773] [ WARNING] ocr_and_match.py:28 - OCR detection failed: tuple index out of range
[2025-09-22 19:05:36,790] [ WARNING] ocr_and_m

  Text detections: 0
  Scale bar detections: 1
  Successful matches: 0
Results saved to: output/ocr_results_10.json

Processing image 11...
  Text detections: 0
  Scale bar detections: 1
  Successful matches: 0
Results saved to: output/ocr_results_11.json

Processing image 12...
  Text detections: 0
  Scale bar detections: 1
  Successful matches: 0
Results saved to: output/ocr_results_12.json

Processing image 13...
  Text detections: 0
  Scale bar detections: 1
  Successful matches: 0
Results saved to: output/ocr_results_13.json

Processing image 14...
  Text detections: 0
  Scale bar detections: 0
  Successful matches: 0
Results saved to: output/ocr_results_14.json

Processing image 15...
  Text detections: 0
  Scale bar detections: 0
  Successful matches: 0
Results saved to: output/ocr_results_15.json

Processing image 16...
  Text detections: 0
  Scale bar detections: 0
  Successful matches: 0
Results saved to: output/ocr_results_16.json

Processing image 17...
  Text detections: 0

## 6. Pixel to Physical Unit Conversion


In [58]:
# Test pixel to physical unit conversion
print("Testing pixel to physical unit conversion...")

# Create converters from successful matches
converters = []
for i, results in enumerate(sample_results):
    if results['matches']:
        converter = create_converter_from_matches(results['matches'])
        if converter:
            converters.append((i+1, converter))
            print(f"\nImage {i+1} converter:")
            scale_info = converter.get_scale_info()
            print(f"  um_per_pixel: {scale_info['um_per_pixel']:.6f}")
            print(f"  mm_per_pixel: {scale_info['mm_per_pixel']:.6f}")
            print(f"  nm_per_pixel: {scale_info['nm_per_pixel']:.6f}")

print(f"\nCreated {len(converters)} converters successfully!")


Testing pixel to physical unit conversion...

Created 0 converters successfully!


In [ ]:
# Demonstrate conversion utilities
if converters:
    print("Demonstrating conversion utilities...")
    
    # Use first converter for demonstration
    img_id, converter = converters[0]
    
    # Test coordinate conversion
    pixel_coords = [(100, 200), (300, 400), (500, 600)]
    mm_coords = converter.convert_coordinates(pixel_coords, 'mm')
    um_coords = converter.convert_coordinates(pixel_coords, 'um')
    
    print(f"\nCoordinate conversion (Image {img_id}):")
    for i, (pixel, mm, um) in enumerate(zip(pixel_coords, mm_coords, um_coords)):
        print(f"  Point {i+1}: {pixel} px -> {mm} mm -> {um} um")
    
    # Test distance conversion
    pixel_distance = 150.0
    mm_distance = converter.convert_distance(pixel_distance, 'mm')
    um_distance = converter.convert_distance(pixel_distance, 'um')
    
    print(f"\nDistance conversion:")
    print(f"  {pixel_distance} px -> {mm_distance:.6f} mm -> {um_distance:.6f} um")
    
    # Test area conversion
    pixel_area = 10000.0  # 100x100 pixels
    mm_area = converter.convert_area(pixel_area, 'mm')
    um_area = converter.convert_area(pixel_area, 'um')
    
    print(f"\nArea conversion:")
    print(f"  {pixel_area} px² -> {mm_area:.12f} mm² -> {um_area:.6f} um²")
    
    # Test bounding box conversion
    bbox_pixel = (50, 100, 200, 150)  # (x, y, w, h)
    bbox_mm = converter.convert_bbox(bbox_pixel, 'mm')
    
    print(f"\nBounding box conversion:")
    print(f"  {bbox_pixel} px -> {bbox_mm} mm")
else:
    print("No converters available for demonstration.")


## 7. Pipeline Summary and Results


In [ ]:
# Create pipeline summary
print("=" * 80)
print("SCALE DETECTION PIPELINE - EXECUTION SUMMARY")
print("=" * 80)

print(f"\n1. DATASET CONVERSION:")
print(f"   ✓ Converted {len(json_files)} JSON annotations to YOLO format")
print(f"   ✓ Created dataset configuration: {yaml_path}")
print(f"   ✓ Validation: {stats['files_with_annotations']} files with annotations")

print(f"\n2. MODEL TRAINING:")
print(f"   ✓ Trained YOLOv8m model for joint detection")
print(f"   ✓ Model saved to: {training_config['output_dir']}")
print(f"   ✓ Exported ONNX model: {exported_path}")

print(f"\n3. ENDPOINT LOCALIZATION:")
print(f"   ✓ Implemented fine-grained endpoint detection")
print(f"   ✓ Tested on sample images")
print(f"   ✓ Visualization saved to: {OUTPUT_DIR}")

print(f"\n4. OCR AND MATCHING:")
print(f"   ✓ Implemented PaddleOCR integration")
print(f"   ✓ Text parsing with unit normalization")
print(f"   ✓ Spatial matching between text and scale bars")
print(f"   ✓ Processed {len(sample_results)} sample images")

print(f"\n5. PIXEL TO PHYSICAL CONVERSION:")
print(f"   ✓ Created {len(converters)} scale converters")
print(f"   ✓ Implemented coordinate, distance, and area conversion")
print(f"   ✓ Support for mm, μm, and nm units")

print(f"\n6. EVALUATION FRAMEWORK:")
print(f"   ✓ Comprehensive evaluation metrics")
print(f"   ✓ Detection, OCR, and scale conversion evaluation")
print(f"   ✓ Report generation and visualization")

print(f"\n7. OUTPUT FILES:")
print(f"   ✓ YOLO dataset: {yolo_output_dir}")
print(f"   ✓ Trained model: {training_config['output_dir']}")
print(f"   ✓ ONNX model: {exported_path}")
print(f"   ✓ OCR results: {OUTPUT_DIR}/ocr_results_*.json")
print(f"   ✓ Visualizations: {OUTPUT_DIR}/*.png")

print(f"\n" + "=" * 80)
print("PIPELINE EXECUTION COMPLETED SUCCESSFULLY!")
print("=" * 80)


In [ ]:
# Test the updated pipeline with YOLO detections
print("Testing updated OCR pipeline with YOLO detections...")

# Load the trained model
best_model_path = 'models/train/weights/best.pt'
model = YOLO(best_model_path)

# Test on a few sample images
test_images = [1, 2, 3, 4, 5]
sample_results = []

for i in test_images:
    image_path = os.path.join(IMAGES_DIR, f"train/{i}.jpg")
    
    if os.path.exists(image_path):
        print(f"\nProcessing image {i}...")
        
        # Load image
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Use YOLO model to detect both scale bars and text labels
        print(f"  Running YOLO detection...")
        yolo_results = model.predict(image, conf=0.25, verbose=False)
        
        # Process YOLO detections with OCR on text label crops
        results = pipeline.process_yolo_detections(image, yolo_results[0])
        
        print(f"  YOLO scale bars: {results.get('yolo_scale_bars', 0)}")
        print(f"  YOLO text labels: {results.get('yolo_text_labels', 0)}")
        print(f"  Text detections: {results['total_text_detections']}")
        print(f"  Scale bar detections: {results['total_bar_detections']}")
        print(f"  Successful matches: {results['successful_matches']}")
        
        # Show matched scales
        for j, match in enumerate(results['matches']):
            print(f"    Match {j+1}: '{match.text.text}' -> {match.text.parsed_value} {match.text.normalized_unit}")
            if match.um_per_pixel:
                print(f"      Scale: {match.um_per_pixel:.6f} um/pixel")
        
        sample_results.append(results)
        
        # Save results
        output_path = os.path.join(OUTPUT_DIR, f"yolo_ocr_results_{i}.json")
        pipeline.save_results(results, output_path)

print(f"\nProcessed {len(sample_results)} images with updated pipeline!")
